In [1]:
import pandas as pd
import numpy as np
import math
import json
import requests
from SPARQLWrapper import SPARQLWrapper, JSON

In [2]:
data = pd.read_csv('data.csv', delimiter=', ')
data = data.drop(columns=[data.columns[0]])
df_day = pd.read_csv('context_day.csv', delimiter=', ')
df_day = df_day.drop(columns=[df_day.columns[0]])
df_place = pd.read_csv('context_place.csv', delimiter=', ')
df_place = df_place.drop(columns=[df_place.columns[0]])
users = data.index.tolist()
#т.к. удаляю столбец для более удобного форматирования данных, нумерация юзеров сдвигается на -1
my_user_number = 19
no_rate = -1
home = 'h'
saturday = 'Sat'
sunday = 'Sun'
print(data)

    Movie 1  Movie 2  Movie 3  Movie 4  Movie 5  Movie 6  Movie 7  Movie 8  \
0        -1        4        1        5       -1       -1       -1        1   
1         4        5        2        3        3        4       -1        5   
2         4        2        5        4        1        5       -1        3   
3         5       -1        2       -1       -1        2        4        3   
4         4        3        3        3        5        3        3        4   
5         2       -1        3        3        3        3        4        2   
6         4        1        4        4        1        4        5        2   
7        -1        5        5        3        3        3        3        3   
8         1        2        3        1        5        3        3        1   
9         5       -1        1       -1        3        3        4        1   
10       -1        5       -1        1        5        2        4        2   
11        5        4        4       -1        3       -1        

In [3]:
tr_data = data.transpose()
#неоцененные фильмы
not_rated_movies = tr_data[tr_data[my_user_number] == no_rate].index
print(tr_data)
print(not_rated_movies)

          0   1   2   3   4   5   6   7   8   9   ...  30  31  32  33  34  35  \
Movie 1   -1   4   4   5   4   2   4  -1   1   5  ...   3   5   5  -1   4   3   
Movie 2    4   5   2  -1   3  -1   1   5   2  -1  ...   5   4   3   5  -1   2   
Movie 3    1   2   5   2   3   3   4   5   3   1  ...   4   5   4  -1   5  -1   
Movie 4    5   3   4  -1   3   3   4   3   1  -1  ...   5   4   4  -1   4   5   
Movie 5   -1   3   1  -1   5   3   1   3   5   3  ...   1   1   2   1   2   5   
Movie 6   -1   4   5   2   3   3   4   3   3   3  ...   3   5   1  -1   2   1   
Movie 7   -1  -1  -1   4   3   4   5   3   3   4  ...   3   5   3   5   3   5   
Movie 8    1   5   3   3   4   2   2   3   1   1  ...   3   4   1   1   1   2   
Movie 9    3   3   5   2   5  -1   2   1   2   1  ...   4   4   3   1   1   1   
Movie 10  -1   2   2   3   2   1  -1   2   5   2  ...   1  -1   3   5   5   5   
Movie 11   5   2   1  -1   3   5   4  -1   1  -1  ...   2   4   5   3  -1   2   
Movie 12   5  -1  -1  -1   2

In [4]:
#поиск sim
sims = {}
for col in tr_data:
    if(col == my_user_number):
        continue
    two_users_rates = data.transpose()[[col, my_user_number]]
    two_users_rates = two_users_rates[(two_users_rates[col] != no_rate) & (two_users_rates[my_user_number] != no_rate)]
    numerator = 0
    u_sum = 0
    v_sum = 0
    denominator = 0
    for i in range(0, two_users_rates.shape[0]):
        u = two_users_rates.iloc[i][col]
        v = two_users_rates.iloc[i][my_user_number]
        numerator += u * v
        u_sum += u ** 2
        v_sum += v ** 2
    denominator = math.sqrt(u_sum) * math.sqrt(v_sum)
    sim = round(numerator / denominator, 3)
    sims[col] = sim
sims = dict(sorted(sims.items(), key = lambda x: x[1], reverse = True))
max_sims = dict(list(sims.items())[:4])
print(max_sims)

{16: 0.899, 22: 0.888, 7: 0.88, 27: 0.879}


In [5]:
#поиск средних оценок пользователей
avg = {}
for col in tr_data:
    avg[col] = round(tr_data[col][tr_data[col] != no_rate].mean(), 3)
print(avg)


{0: 2.952, 1: 3.292, 2: 2.8, 3: 3.143, 4: 3.308, 5: 3.16, 6: 2.88, 7: 3.292, 8: 2.808, 9: 2.72, 10: 3.125, 11: 3.846, 12: 3.154, 13: 3.0, 14: 3.375, 15: 3.407, 16: 3.0, 17: 2.704, 18: 3.125, 19: 3.182, 20: 2.96, 21: 3.217, 22: 3.391, 23: 2.815, 24: 2.76, 25: 2.92, 26: 3.333, 27: 2.923, 28: 3.2, 29: 2.964, 30: 3.074, 31: 3.0, 32: 2.769, 33: 3.042, 34: 3.174, 35: 3.125, 36: 3.25, 37: 3.0, 38: 2.68, 39: 2.963}


In [6]:
#рассчёт оценок неоцененных фильмов
rates = {}
for movie in not_rated_movies:
    numer = 0
    denom = 0
    sim_sum_abs = 0
    for user in max_sims.keys():
        #если кто-то из ближайших 4х соседей не оценил фильм - берем следующего
        if (tr_data.loc[movie, user] == no_rate):
            continue
        sim_sum_abs += abs(max_sims[user])
        numer += max_sims[user] * (tr_data.loc[movie, user] - avg[user])
    rate = round(avg[my_user_number] + numer / sim_sum_abs, 3)
    rates[movie] = rate
print(rates)

{'Movie 2': 2.275, 'Movie 5': 3.785, 'Movie 6': 3.118, 'Movie 8': 3.982, 'Movie 10': 1.78, 'Movie 17': 3.783, 'Movie 21': 2.737, 'Movie 25': 3.278}


In [7]:
#ключ - фильм, который не посмотрел пользователь
#значение - список пользователей, которые посмотрели фильм в выходной дома
rec_movies = {}
for movie in not_rated_movies:
    home_viewers = df_place[df_place[movie] == home].index.tolist()
    weekend_movies = df_day[(df_day[movie] == saturday) | (df_day[movie] == sunday)].index.tolist()
    rec_movies[movie] = list(set(home_viewers) & set(weekend_movies))
print(rec_movies)

{'Movie 2': [33, 29, 17, 7], 'Movie 5': [1, 34, 11, 20, 28], 'Movie 6': [3, 28], 'Movie 8': [17, 12], 'Movie 10': [33, 29, 21], 'Movie 17': [8, 24], 'Movie 21': [27, 14], 'Movie 25': [28, 39]}


In [8]:
#поиск рекоммендуемого фильма
recs = {}
for movie in rec_movies.keys():
    avg_rate = 0
    for user in rec_movies[movie]:
        avg_rate += (avg[user] - tr_data.loc[movie, user]) * sims[user]
    if(len(rec_movies[movie]) != 0):
        avg_rate = round(avg_rate / len(rec_movies[movie]), 3)
        recs[movie] = avg_rate   
    else:
        recs[movie] = float('-inf')
print(recs)
rec_film = max(recs, key=recs.get)
print(rec_film)

{'Movie 2': -0.855, 'Movie 5': 0.583, 'Movie 6': 0.987, 'Movie 8': -1.217, 'Movie 10': -0.231, 'Movie 17': -0.585, 'Movie 21': 1.414, 'Movie 25': -0.745}
Movie 21


In [9]:
data ={}
result = json.dumps({'user': my_user_number + 1,'part_1': rates, '2': rec_film}, indent = 4)
result = {
    'user': my_user_number + 1,
    'part_1': rates,
    'part_2': rec_film
}
with open('result.json', 'w') as outfile:
    json.dump(result, outfile, indent = 4)
print(result)

{'user': 20, 'part_1': {'Movie 2': 2.275, 'Movie 5': 3.785, 'Movie 6': 3.118, 'Movie 8': 3.982, 'Movie 10': 1.78, 'Movie 17': 3.783, 'Movie 21': 2.737, 'Movie 25': 3.278}, 'part_2': 'Movie 21'}


In [10]:
#task 3.2
df = pd.read_csv("movie_names.csv", delimiter=', ', header = None)
movie_names = df.loc[df[0].isin(not_rated_movies.values)]
movie_names = pd.Series(movie_names[1].values, index=movie_names[0]).to_dict()
movie_names

{'Movie 2': 'The Godfather',
 'Movie 5': '12 Angry Men',
 'Movie 6': "Schindler's List",
 'Movie 8': 'Pulp Fiction',
 'Movie 10': 'Fight Club',
 'Movie 17': 'Goodfellas',
 'Movie 21': 'Se7en',
 'Movie 25': 'Life Is Beautiful'}

In [11]:
API_ENDPOINT = "https://www.wikidata.org/w/api.php"

movie_ids = {}
for movie in movie_names:
    expected_label = movie_names[movie]
    params = {
        'action' : 'wbsearchentities',
        'format' : 'json',
        'language' : 'en',
        'search': expected_label
    }
    res = requests.get(API_ENDPOINT, params = params)
    json_search = res.json()['search']
    film = {}
    for i in range(len(json_search)):
        if "film" in json_search[i]['description']:
            film = json_search[i]
            break
    movie_ids[movie] = film['id']
movie_ids

{'Movie 2': 'Q47703',
 'Movie 5': 'Q2345',
 'Movie 6': 'Q483941',
 'Movie 8': 'Q104123',
 'Movie 10': 'Q190050',
 'Movie 17': 'Q42047',
 'Movie 21': 'Q190908',
 'Movie 25': 'Q19355'}

In [14]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

movie_results_df = {}
for movie in movie_ids:
    id = movie_ids[movie]
    sparql_query = """

select DISTINCT ?movie ?movieLabel
WHERE {
  {
  SELECT ?movie WHERE {
  wd:""" + id + """ wdt:P136 ?myGenre .
  wd:""" + id + """ wdt:P272 ?myStudio .
  
  ?movie wdt:P136 ?genre filter(?myGenre != ?genre) .
  ?movie wdt:P272 ?studio filter(?myStudio = ?studio) .
    }
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".} .
}
 
"""
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    if(len(results['results']['bindings']) == 0):
        movie_results_df[movie] = pd.DataFrame(columns=['movie', 'movieLabel'])
    else:
        results_df = pd.json_normalize(results['results']['bindings'])
        movie_results_df[movie] = results_df[['movie.value', 'movieLabel.value']]

for movie in movie_results_df:
    if(len(movie_results_df[movie]) == 0):
        print(movie,
            "\nQuery results number: 0\n")
    else:
        print(
            movie,
            "\nQuery results number:", len(movie_results_df[movie]), "\n",
            movie_results_df[movie].head(), "\n"
        )

Movie 2 
Query results number: 1123 
                                movie.value  \
0  http://www.wikidata.org/entity/Q1141257   
1  http://www.wikidata.org/entity/Q1138932   
2  http://www.wikidata.org/entity/Q1132575   
3  http://www.wikidata.org/entity/Q1141493   
4  http://www.wikidata.org/entity/Q1142862   

                       movieLabel.value  
0  I Married a Monster from Outer Space  
1                              Grease 2  
2            An Officer and a Gentleman  
3                           The Heiress  
4                             King Kong   

Movie 5 
Query results number: 0

Movie 6 
Query results number: 101 
                               movie.value            movieLabel.value
0   http://www.wikidata.org/entity/Q32433                     Lincoln
1   http://www.wikidata.org/entity/Q11621  E.T. the Extra-Terrestrial
2   http://www.wikidata.org/entity/Q45839         Memoirs of a Geisha
3   http://www.wikidata.org/entity/Q51416                Men in Black
4  http://